In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter

In [51]:
df = pd.read_excel(io='data/dataframe_elections.xlsx')
df_gagnant = pd.read_excel(io='data/histo_elus.xlsx')

In [88]:
depart_frontaliers = ["PAS-DE-CALAIS","NORD", "AISNE", "ARDENNES", "MEUSE",
                      "MEURTHE-ET-MOSELLE", "MOSELLE", "BAS-RHIN", "HAUT-RHIN","TERRITOIRE DE BELFORT", "DOUBS", "JURA", "AIN", "HAUTE-SAVOIE", \
                      "SAVOIE", "HAUTES-ALPES", "ALPES-DE-HAUTE-PROVENCE", "ALPES-MARITIMES", \
                      "PYRENEES-ATLANTIQUES", "HAUTES-PYRENEES", "HAUTE-GARONNE",\
                      "ARIEGE", "PYRENEES-ORIENTALES", "AUDE", "HERAULT", "GARD",\
                     "BOUCHES-DU-RHONE", "VAR"]
depart_OM = ["GUADELOUPE", "GUYANE", "MARTINIQUE", "LA REUNION", "MAYOTTE"]
depart_corse = ["CORSE", "HAUTE-CORSE", "CORSE-DU-SUD"]
region_parisienne = ["PARIS","SEINE-ET-MARNE", "YVELINES", "ESSONNE", "HAUTS-DE-SEINE", "SEINE-SAINT-DENIS",\
                    "VAL-DE-MARNE", "VAL-D'OISE"]



In [89]:
df["dep"] = df["dep"].map(lambda x : x.upper())

In [90]:
df["depart_frontalier"] = [1 if dep in depart_frontaliers else 0 for dep in df["dep"]]
df["depart_OM"] = [1 if dep in depart_OM else 0 for dep in df["dep"]]
df["depart_CORSE"] = [1 if dep in depart_corse else 0 for dep in df["dep"]]
df["region_parisienne"] = [1 if dep in region_parisienne else 0 for dep in df["dep"]]

In [96]:
df[["dep", "depart_frontalier", "depart_OM", "depart_CORSE", "region_parisienne"]].drop_duplicates().to_excel("data/geographie_depart.xlsx")

In [64]:
df_gagnant["gagnant"] = 1
df = df.merge(df_gagnant[["Nom", "Prénom", "gagnant", "Année", "Circo"]], left_on=["nom", "prenom", "an", "circo"], \
         right_on=["Nom", "Prénom", "Année", "Circo"], how="left").drop(["Nom", "Prénom", "Année", "Circo"], axis=1)\
         

In [65]:
df.shape

(36919, 69)

In [ ]:
to_dummifiy = ["c_dep", "circo"]
to_drop = ["dep", "code", "circo_parti"]
to_standardize = ["inscrits", "revenus_q1", "revenus_med", "revenus_q3", \
               "ecart_revenus"]
proba_var = ["etrangers", "part_impose", "chom_tot", "chom_tot_evol_5", \
             "chom_jeunes", "chom_jeunes_evol_5", "chom_adultes", "chom_adultes_evol_5",\
             "chom_seniors", "chom_seniors_evol_5", "p_agri", "p_commercants", "p_cadres",\
             "p_intermed", "p_employes", "p_ouvriers", "d_brevet", "d_bep", "d_bac", "d_sup"]

In [66]:
a = [5,2,63,5]

In [67]:
a.remove(63)

In [71]:
print(12%5)

2


In [75]:
print(12/5)

2.4


In [47]:
df.columns

Index(['an', 'c_dep', 'dep', 'circo', 'code', 'inscrits', 'etrangers',
       'part_impose', 'revenus_q1', 'revenus_med', 'revenus_q3',
       'ecart_revenus', 'chom_tot', 'chom_tot_evol_5', 'chom_jeunes',
       'chom_jeunes_evol_5', 'chom_adultes', 'chom_adultes_evol_5',
       'chom_seniors', 'chom_seniors_evol_5', 'p_agri', 'p_commercants',
       'p_cadres', 'p_intermed', 'p_employes', 'p_ouvriers', 'd_brevet',
       'd_bep', 'd_bac', 'd_sup', 'circo_parti', 'circo_nuance',
       'circo_nuance_groupe', 'circo_bloc', 'circo_leg_meme_nuance',
       'nb_candidats_meme_bloc', 'score_nuance_groupe_prec_leg',
       'score_bloc_prec_leg', 'taux_vote_leg', 'score_leg_exg', 'score_leg_g',
       'score_leg_c', 'score_leg_d', 'score_leg_exd', 'score_leg_div',
       'circo_nuance_groupe_pres', 'circo_pres_meme_nuance',
       'score_nuance_groupe_pres', 'score_bloc_pres', 'taux_vote_pres',
       'score_pres_exg', 'score_pres_g', 'score_pres_c', 'score_pres_d',
       'score_pres_exd', 

In [48]:
df.isnull().sum()

an                              0
c_dep                           0
dep                             0
circo                           0
code                            0
inscrits                     6536
etrangers                    6714
part_impose                  7443
revenus_q1                   7443
revenus_med                  7443
revenus_q3                   7443
ecart_revenus                7443
chom_tot                     6714
chom_tot_evol_5              6714
chom_jeunes                  6714
chom_jeunes_evol_5           6714
chom_adultes                 6714
chom_adultes_evol_5          6714
chom_seniors                 6714
chom_seniors_evol_5          6714
p_agri                       7034
p_commercants                7034
p_cadres                     7034
p_intermed                   7034
p_employes                   7034
p_ouvriers                   7034
d_brevet                     7034
d_bep                        7034
d_bac                        7034
d_sup         

In [6]:
df.drop(["Code", "département", 'Score du candidat aux législatives précédentes (si applicable)',
       'Le candidat est-il le député sortant ? 0 ou 1',
       'Parti du député sortant de la circonscription',
       'Score du parti aux présidentielles dans la circonscription',
       'Score du parti aux législatives précédentes (cumul si alliances)',
       'Nombres de candidats en lice (ou leur score de parti cumulé)',
       'Nombre de candidats de la même nuance en lice', 'Député sortant ?', 
       "Nom candidat", "Prénom candidat", "Etiquette liste", 'voix', '% voix'], axis=1, inplace=True)

In [7]:
df[['Part Agriculteurs',
       "Part Artisans, Commerçants, Chefs d'Entreprise",
       'Part Cadres et Professions intellectuelles sup',
       'Part Professions intermédiaires', 'Part Employés', 'Part Ouvriers',
       'Part de diplômés A (Brevet)', 'Part de diplômés B (CAP, BEP)',
       'Part de diplômés C (Bac)', 'Part de diplômés D (Etudes Sup)']] = df[['Part Agriculteurs',
       "Part Artisans, Commerçants, Chefs d'Entreprise",
       'Part Cadres et Professions intellectuelles sup',
       'Part Professions intermédiaires', 'Part Employés', 'Part Ouvriers',
       'Part de diplômés A (Brevet)', 'Part de diplômés B (CAP, BEP)',
       'Part de diplômés C (Bac)', 'Part de diplômés D (Etudes Sup)']].apply(lambda x : x.fillna(x.mean()))

### On vire les lignes qui restent nan :

In [8]:
df.dropna(axis=1, inplace=True)

### Dummies

In [9]:
df = pd.get_dummies(df, columns=["Année", "Code département", "Sexe candidat"])

In [10]:
df = pd.get_dummies(df, columns=["nuance"])

In [11]:
df["Accès second tour"].unique()

array(['N', 'O'], dtype=object)

In [12]:
df["Accès second tour"] = df["Accès second tour"].map(lambda x : 0 if x=="N" else 1)

### Standardisation  

Flemme :)

### Modélisation

In [13]:
df.shape

(22687, 144)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(df.drop("Accès second tour", axis=1), df["Accès second tour"], test_size=0.33, random_state=42)

#### Dealing with the imbalanced dataset

Taux de 1 dans le dataset : 

In [15]:
np.sum(df["Accès second tour"]==1)*100/(np.sum(df["Accès second tour"]==1)+np.sum(df["Accès second tour"]==0))

13.853748842949708

Poids à donner aux 1 :

In [16]:
100/13

7.6923076923076925

In [17]:
sample_weight = np.array([8 if i == 1 else 1 for i in y_train])

In [37]:
clf = RandomForestClassifier(n_estimators=20, max_depth=80)
clf.fit(X_train, y_train, sample_weight=sample_weight)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=80, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [38]:
print(classification_report(clf.predict(X_test), y_test))

             precision    recall  f1-score   support

          0       0.97      0.96      0.96      6537
          1       0.74      0.80      0.77       950

avg / total       0.94      0.94      0.94      7487



** Feat importance **

In [39]:
pd.DataFrame(clf.feature_importances_, list(df.drop("Accès second tour", axis=1).columns), columns=["col"]).sort_values("col", ascending=False).head(20)

,col
nuance_UMP,0.159962
nuance_SOC,0.132782
nuance_EXG,0.094744
nuance_DIV,0.068790
Inscrits,0.065299
circonscription,0.054475
nuance_ECO,0.045317
nuance_DVD,0.030101
nuance_EXD,0.028476
nuance_FRN,0.020169


In [40]:
X_test["pred"] = clf.predict(X_test)

In [45]:
X_test[X_test["pred"]==1][['nuance_COM',
'nuance_DIV',
'nuance_DVD',
'nuance_DVG',
'nuance_ECO',
'nuance_EXD',
'nuance_EXG',
'nuance_FDG',
'nuance_FRN',
'nuance_MODEM',
'nuance_MODEM-',
'nuance_MPF',
'nuance_NouvC',
'nuance_PRG',
'nuance_PRV',
'nuance_PSLE',
'nuance_SOC',
'nuance_UDF',
'nuance_UMP']].sum()

nuance_COM         4
nuance_DIV         3
nuance_DVD        17
nuance_DVG        16
nuance_ECO        11
nuance_EXD         0
nuance_EXG         0
nuance_FDG         0
nuance_FRN         9
nuance_MODEM       0
nuance_MODEM-      0
nuance_MPF         0
nuance_NouvC       1
nuance_PRG        35
nuance_PRV         0
nuance_PSLE        1
nuance_SOC       392
nuance_UDF         7
nuance_UMP       454
dtype: int64